In [1]:
import sounddevice as sd
import numpy as np
import tkinter as tk
from tkinter import ttk
import queue
import threading
import time

# Define loudness thresholds in dB SPL
NOT_SPEAKING_THRESHOLD = 10  # Below 10 dB considered silent
SOFT_THRESHOLD = 25         # 10-20 dB SPL is considered soft
ADEQUATE_THRESHOLD = 44     
LOUD_THRESHOLD = 80          

# Parameters for audio capture
SAMPLE_RATE = 44100
BLOCK_DURATION = 0.5
CHANNELS = 1

class LoudnessGUI:
    def __init__(self, root):
        self.root = root
        root.title("Interview Loudness Monitor")
        root.geometry("400x300")
        
        self.main_frame = ttk.Frame(root, padding=20)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        self.status_label = ttk.Label(self.main_frame, font=('Helvetica', 24, 'bold'))
        self.status_label.pack(pady=10)
        
        self.db_label = ttk.Label(self.main_frame, font=('Helvetica', 18))
        self.db_label.pack(pady=5)
        
        self.canvas = tk.Canvas(self.main_frame, height=30, bg='white')
        self.canvas.pack(fill=tk.X, pady=10)
        self.progress = self.canvas.create_rectangle(0, 0, 0, 30, fill='grey')
        
        self.advice_label = ttk.Label(self.main_frame, font=('Helvetica', 12), wraplength=350)
        self.advice_label.pack(pady=10)
        
        self.data_queue = queue.Queue()
        self.start_audio_thread()
        self.root.after(100, self.update_ui)
    
    def start_audio_thread(self):
        def audio_thread():
            block_size = int(SAMPLE_RATE * BLOCK_DURATION)
            with sd.InputStream(
                callback=lambda i, f, t, s: self.audio_callback(i, f, t, s),
                channels=CHANNELS,
                samplerate=SAMPLE_RATE,
                blocksize=block_size
            ):
                while True:
                    time.sleep(0.1)
        
        thread = threading.Thread(target=audio_thread, daemon=True)
        thread.start()
    
    def audio_callback(self, indata, frames, time, status):
        db_spl = calculate_db_spl(indata)
        
        if db_spl < NOT_SPEAKING_THRESHOLD:
            classification = "NOT SPEAKING"
            advice = "No speech detected"
            color = 'grey'
        elif db_spl < SOFT_THRESHOLD:
            classification = "TOO SOFT"
            advice = "Speak louder or move closer to the microphone"
            color = 'red'
        elif db_spl < ADEQUATE_THRESHOLD:
            classification = "ADEQUATE"
            advice = "Good volume for interview"
            color = 'green'
        else:
            classification = "TOO LOUD"
            advice = "Speak softer or move away from the microphone"
            color = 'red'
        
        self.data_queue.put((db_spl, classification, advice, color))
    
    def update_ui(self):
        try:
            db_spl, classification, advice, color = self.data_queue.get_nowait()
            
            self.status_label.config(text=classification, foreground=color)
            self.db_label.config(text=f"{db_spl:.1f} dB SPL")
            
            # Calculate progress bar width with clamping
            bar_width = (db_spl / LOUD_THRESHOLD) * 400
            bar_width = max(0, min(bar_width, 400))  # Prevent negative/overflow values
            self.canvas.coords(self.progress, 0, 0, bar_width, 30)
            self.canvas.itemconfig(self.progress, fill=color)
            
            self.advice_label.config(text=advice)
            
        except queue.Empty:
            pass
        
        self.root.after(50, self.update_ui)

def calculate_db_spl(audio_data):
    if len(audio_data) == 0:
        return -100
    rms = np.sqrt(np.mean(audio_data**2))
    return 20 * np.log10(rms + 1e-6) + 70  # Calibrated SPL conversion

if __name__ == "__main__":
    root = tk.Tk()
    app = LoudnessGUI(root)
    root.mainloop()
